In [3]:
import tdx

In [4]:
tdx

<module 'tdx' from 'D:\\download\\jupyter-ide\\src\\Lib\\site-packages\\tdx\\__init__.py'>

In [5]:
import tdx

In [6]:
tdx

<module 'tdx' from 'D:\\download\\jupyter-ide\\src\\Lib\\site-packages\\tdx\\__init__.py'>

In [9]:
import tdx

def Initialize(ctx):
    ctx.log.Trace('Trace')
    ctx.log.Debug('Debug')
    ctx.log.Info('Info')
    ctx.log.Notice('Notice')
    ctx.log.Warn('Warn')
    ctx.log.Error('Error')
    ctx.log.Fatal('Fatal')

def Main():
    ctxtype = tdx.ANALYSE # 分析统计
    #ctxtype = tdx.BACKTEST # 回测
    #ctxtype = tdx.TRADE # 交易
    ctx = tdx.Context(type = ctxtype, sdate = 20220101, edate = 20220201,
                      logfilelevel = tdx.LTRACE, 
                      logconsolelevel = tdx.LTRACE,
                      logappend = False)
    if ctx.type == tdx.BACKTEST:
        pass
    elif ctx.type == tdx.TRADE:
        pass
    elif ctx.type == tdx.ANALYSE:
        pass
    tdx.Run(Initialize, ctx)

Main()

In [11]:
import tdx

def OnDelayCheckWT(ctx):
    wt = tdx.WTOrder(tdx.WTOrder, tdx.KDay.close)
    for it in wt:
        wtno = it.data[0] # 委托编号
        #success, wtno = tdx.COrder(wtno) # 没有成交则撤单

def OnKDayBar(ctx): 
    ### 持仓列表(tdx.Pool)
    position = tdx.Position()
    if position: # 存在持仓
        for it in position:
            kline = tdx.KLine(it.code, it.market, num = 20, fq = tdx.FQ_N)
            if -1 == tdx.CROSS(tdx.MA(kline.close, 5), tdx.MA(kline.close, 10), final = True): # 下破
                kline = tdx.KLine(it.code, it.market, num = 1)
                success, wtno = tdx.SOrder(it.code, it.market, kline.close[0], it.data[3]) # 委托卖出
                if success:
                    ctx.log.Info('S-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托成功，委托编号：' + wtno + ' 委托数量：' + str(it.data[1]) + ' 委托价格：' + str(kline.close[0])\
                        + ' 卖出原因：均线下破')
                else:
                    ctx.log.Error('S-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托失败，失败原因：' + wtno)

    else: # 不存在持仓
        # 沪深300(换手率>2% & 高开1%)(应该要剔除涨停股)
        #print(tdx.GZ("002049", tdx.SZ))
        #print(tdx.KLine("002049", tdx.SZ, num = 1))
        cond = (tdx.GZ.hsl >= 2.0) & (((tdx.KDay.open - tdx.KDay.preclose) / tdx.KDay.preclose) * 100 > 1.0)
        pool_hs300 = tdx.Pool('399300', tdx.SZ).Select(cond).Panel(tdx.GZ.hsl)
        gk = pool_hs300(((tdx.KDay.open - tdx.KDay.preclose) / tdx.KDay.preclose) * 100) #高开1%
        l = []
        for it in pool_hs300:
            kline = tdx.KLine(it.code, it.market, num = 20, fq = tdx.FQ_N)
            if 1 == tdx.CROSS(tdx.MA(kline.close, 5), tdx.MA(kline.close, 10), final = True): # 上穿
                l.append((it.code, it.market))
        if l:
            pool = tdx.Pool(l).Sort(tdx.GZ.hsl, asc = False) # 换手率从大到小排序
            code, market, data = pool[0] # 排名第一的
            kline = tdx.KLine(code, market, num = 1)
            num = int(ctx.account.kyye / kline.close[0] / 100) * 100 # 买入数量(100的倍数)
            success, wtno = tdx.BOrder(code, market, kline.close[0], num) # 委托买入
            if success:
                ctx.log.Info('B-' + code + ('.SH' if market==tdx.SZ else '.SZ') + ' 委托成功，委托编号：' + wtno + ' 委托数量：' + str(num) + ' 委托价格：' + str(kline.close[0])\
                    + ' 买入原因：均线上穿')
                ctx.event.Delay(OnDelayCheckWT, 120000) # 2分钟后检查一次是否成交
            else:
                ctx.log.Error('B-' + code + ('.SH' if market==tdx.SZ else '.SZ') + ' 委托失败，失败原因：' + wtno)


def OnKCloseLookUpWTOrders(ctx):
    #################################################################
    # 收盘检查今日委托单
    #################################################################
    wt = tdx.WTOrder()
    for it in wt:
        if tdx.WTUNHAPPEN == it.data[2] or tdx.WTFAILED == it.data[2] or tdx.WTCLEAR == it.data[2]:
            if tdx.WTBUY == it.data[1]:
                ctx.log.Info('B-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托未成交，' + '委托数量：' + str(it.data[3]) + ' 委托价格：' + '%.2f'%it.data[4] + ' 委托时间：' + str(it.data[7]))
            elif tdx.WTSELL == it.data[1]:
                ctx.log.Info('S-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托未成交，' + '委托数量：' + str(it.data[3]) + ' 委托价格：' + '%.2f'%it.data[4] + ' 委托时间：' + str(it.data[7]))
        elif tdx.WTCANCEL == it.data[2]:
            if tdx.WTBUY == it.data[1]:
                ctx.log.Info('B-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托撤单，' + '委托数量：' + str(it.data[3]) + ' 委托价格：' + '%.2f'%it.data[4] + ' 委托时间：' + str(it.data[7]) + ' 撤单时间：' + str(it.data[8]))
            elif tdx.WTSELL == it.data[1]:
                ctx.log.Info('S-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托撤单，' + '委托数量：' + str(it.data[3]) + ' 委托价格：' + '%.2f'%it.data[4] + ' 委托时间：' + str(it.data[7]) + ' 撤单时间：' + str(it.data[8]))
        elif tdx.WTHAPPEN == it.data[2]:
            if tdx.WTBUY == it.data[1]:
                ctx.log.Info('B-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托成交，' + '委托数量：' + str(it.data[3]) + ' 委托价格：' + '%.2f'%it.data[4] + ' 委托时间：' + str(it.data[7]) + ' 成交时间：' + str(it.data[8]) + ' 发生金额：' + '%.2f'%it.data[5])
            elif tdx.WTSELL == it.data[1]:
                ctx.log.Info('S-' + it.code + ('.SH' if it.market==tdx.SZ else '.SZ') + ' 委托成交，' + '委托数量：' + str(it.data[3]) + ' 委托价格：' + '%.2f'%it.data[4] + ' 委托时间：' + str(it.data[7]) + ' 成交时间：' + str(it.data[8]) + ' 发生金额：' + '%.2f'%it.data[5])


def OnFinish(ctx):
    ctx.log.Info('%s' % ctx.account)
    yk = ctx.account.sz +  ctx.account.ye -  ctx.account.zrzj # 盈亏 = 市值 + 余额 - 转入资金
    yk_rate = yk / ctx.account.zrzj * 100
    if yk < 0:
        ctx.log.Info('亏损：%.2f  %.2f%%' % (yk, yk_rate))
    else:
        ctx.log.Info('盈利：%.2f  %.2f%%' % (yk, yk_rate))


def Initialize(ctx):
    #################################################################
    '''
    均线策略
    '''
    #################################################################
    ctx.event.KBar(OnKDayBar, ktype = tdx.K_DAY, closetime = 145500)
    ctx.event.KClose(OnKCloseLookUpWTOrders)
    ctx.event.Finish(OnFinish)


def main():
    ctx = tdx.Context(type = tdx.BACKTEST, 
                      sdate = 20210604,
                      edate = 20211128, 
                      logfilelevel = tdx.LTRACE,
                      logconsolelevel = tdx.LINFO,
                      logappend = True)
    if ctx.type == tdx.BACKTEST: ### 回测 设置费率
    #    ctx.account.zrzj = 100000 # 转入资金(初始化资金)
    #    ctx.cost.buytaxmin = 0 # 最低买入佣金 默认5元
    #    ctx.cost.selltaxmin = 0 # 最低卖出佣金 默认5元
     #   ctx.cost.buytaxrate = 0.00000 # 买入佣金率 默认 0.0003
     #   ctx.cost.selltaxrate = 0.00000 # 卖出佣金率 默认 0.0003
     #   ctx.cost.buystamprate = 0 # 买入印花费率 默认 0.000
    #    ctx.cost.sellstamprate = 0.000 # 卖出印花费率 默认 0.001
     #   ctx.cost.buytransferrate = 0.00000 # 买入过户费率 默认 0.00002
    #    ctx.cost.selltransferrate = 0.00000 # 卖出过户费率 默认 0.00002
    #    ctx.cost.fhrate = 0.2 # 股票分红税率 默认 0.20
    #    ctx.cost.sgrate = 0.2 # 送股税率 默认 0.20
        pass
    elif ctx.type == tdx.TRADE: ### 交易 登录账号
        pass
    elif ctx.type == tdx.ANALYSE: ### 统计分析
        pass

    tdx.Run(Initialize, ctx)
    tdx.CloseCache()
    print('#######################################')

if __name__ == "__main__":
    main()

#######################################
